In [ ]:
import torch
from utils import *
from dataset import ImageDataset
from models.baselines.baseline_vanilla_unet import UNet

In [ ]:
checkpoint = torch.load("../checkpoints/kaggle_0.90_weights_vanillaunet.pt", map_location=torch.device('cpu'))

In [ ]:
model = UNet().to("cpu")
model.load_state_dict(checkpoint["model_state_dict"])

In [ ]:
import os
from glob import glob

from tqdm import tqdm
from utils import *
from consts import *
import cv2
import numpy as np
from datetime import datetime

device = "cpu"

# predict on test set
test_path = os.path.join("../data/test", "images")
test_filenames = glob(test_path + "/*.png")
test_images = load_all_from_path(test_path)
batch_size = test_images.shape[0]
size = test_images.shape[1:3]
# we also need to resize the test images. This might not be the best ideas depending on their spatial resolution.
log("Resizing test images...")
test_images = np.stack([cv2.resize(img, dsize=(384, 384)) for img in test_images], 0)
test_images = test_images[:, :, :, :3]
test_images = np_to_tensor(np.moveaxis(test_images, -1, 1), device)

In [ ]:
log("Making predictions...")

test_pred = [model(t).detach().cpu().numpy() for t in tqdm(test_images.unsqueeze(1))]
test_pred = np.concatenate(test_pred, 0)
test_pred = np.moveaxis(test_pred, 1, -1)  # CHW to HWC
test_pred = np.stack([cv2.resize(img, dsize=size) for img in test_pred], 0)  # resize to original shape
# now compute labels
test_pred = test_pred.reshape((-1, size[0] // PATCH_SIZE, PATCH_SIZE, size[0] // PATCH_SIZE, PATCH_SIZE))
test_pred = np.moveaxis(test_pred, 2, 3)
test_pred = np.round(np.mean(test_pred, (-1, -2)) > CUTOFF)


In [ ]:
log(f"Test predictions shape: {test_pred.shape}")
now = datetime.now()
t = now.strftime("%Y-%m-%d_%H:%M:%S")
os.makedirs("submissions", exist_ok=True)
create_submission(
    test_pred,
    test_filenames,
    submission_filename=f"../../submissions/baseline_unet_submission_{t}.csv",
)